In [ ]:
#!pip install gradio==4.44.0 requests==2.32.3

In [ ]:
import gradio as gr
import requests
import json


In [ ]:
model_name = "mistralai/mistral-small-3.2-24b-instruct:free"
api_key = ""
API_URL = "https://openrouter.ai/api/v1/chat/completions"

In [ ]:
def apply_settings(user_api_key, chosen_model):
  global api_key, model_name
  api_key = user_api_key
  model_name = chosen_model or model_name


In [ ]:
def stream_chat_reply(user_message, conversation_history=[]):
  if not api_key:
    return "Please provide your Open Router API key!"

  messages = [
      {"role": "system", "content": "Your useful coding tutor!"}
  ]

  for past_user_message, past_bot_message in conversation_history:
    if past_user_message:
      messages.append({"role": "user", "content": past_user_message})
    if past_bot_message:
      messages.append({"role": "assistant", "content": past_bot_message})
  messages.append({"role": "user", "content": user_message})
  headers = {
      "Authorization": f"Bearer {api_key}",
      "Content-Type": "application/json",
      "Accept": "text/event-stream",
  }

  body = {
      "model": model_name,
      "messages": messages,
      "temperature": 0.7,
      "stream": True,
  }

  with requests.post(API_URL, headers=headers, json=body, stream=True, timeout=300) as response:
    partial_text = ""

    for line in response.iter_lines(decode_unicode=True):
      if not line or not line.startswith("data:"):
        continue
      data = line[5:].strip()
      if data == "[DONE]":
        break
      if not data.startswith("{"):
        continue

      chunk = json.loads(data)
      delta = chunk.get("choices", [{}])[0].get("delta", {})
      token = delta.get("content")
      if token:
        partial_text += token
        yield partial_text

In [ ]:
with gr.Blocks() as demo:
  gr.Markdown("# chucky chatbot")

  with gr.Row():
    api_key_box = gr.Textbox(
        label="Open Router API Key🗝️",
        placeholder="Paste your Open Router API key here",
        type="password"
    )
    model_box = gr.Dropdown(
        label="🤖 Model",
        choices=[
            "mistralai/mistral-small-3.2-24b-instruct:free",
            "mistralai/mistral-7b-instruct:free"
        ],
        value="mistralai/mistral-small-3.2-24b-instruct:free",
        interactive=True
    )
    apply_button = gr.Button("Apply")
  status = gr.Markdown("Paste Your API Key Above, then click Apply")
  apply_button.click(apply_settings, [api_key_box, model_box], status)
  gr.Markdown("Ask me anything!")
  gr.ChatInterface(fn=stream_chat_reply).queue()
demo.launch(share=True, debug=True)